### 라벨링

- 모델1, crash - ego : 0(충돌안함), 1(충돌-나랑), 2(충돌-나아님)
- 모델2, 날씨 : 0(정상), 1(눈), 2(비)
- 모델3. 시간 : 0(낮), 1(밤)

모델1 아이디어
- crash_ego 컬럼 생성, 라벨 부여
- 1,2 데이터 증강없이 1차 시도
- c3d 모델 기반 : c3d baseline, 이후 r3d 적용

모델2 아이디어
- 라벨 달기
- 프레임별로 라벨 달기 : 연산량 부하를 고려해 5프레임 단위로 추출
- image clf 진행 : resnet50

모델3 아이디어
- 라벨 달기
- 프레임별로 라벨 달기 : 연산량 부하를 고려해 5프레임 단위로 추출
- image clf 진행 : resnet50

In [89]:
import pandas as pd

In [92]:
# df = pd.read_csv('./data/train.csv')
# df[['crash_ego','weather','timing']] = ''

# for i in range(len(df)):
#     label = df.loc[i, 'label']
    
#     if (label == 0):
#         df.loc[i, 'crash_ego'] = 0 # 충돌 안함
#     elif ((label > 0) and (label <7)):
#         df.loc[i, 'crash_ego'] = 1 # 충돌 - 나랑충돌
#     else:
#         df.loc[i, 'crash_ego'] = 2 # 충돌 - 나랑 충돌하지 않음
      
#     if (label==0):
#         df.loc[i, 'weather'] = 'Na'
#     elif ((label==1) or (label==2) or (label==7) or (label==8)):
#         df.loc[i, 'weather'] = 0 # 정상
#     elif ((label==3) or (label==4) or (label==9) or (label==10)):
#         df.loc[i, 'weather'] = 1 # 눈
#     else:
#         df.loc[i, 'weather'] = 2 # 비
    
#     if (label==0):
#         df.loc[i, 'timing'] = 'Na'
#     elif ((label==1) or (label==3) or (label==5) or (label==7) or (label==9) or (label==11)):
#         df.loc[i, 'timing'] = 0 # 낮
#     else:
#         df.loc[i, 'timing'] = 1 # 밤

# df.to_csv('./data/train_new_label.csv', index=False)

In [110]:
df = pd.read_csv('./data/train_new_label.csv')
df

,sample_id,video_path,label,crash_ego,weather,timing
0,TRAIN_0000,./train/TRAIN_0000.mp4,7,2,0,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,7,2,0,0
2,TRAIN_0002,./train/TRAIN_0002.mp4,0,0,Na,Na
3,TRAIN_0003,./train/TRAIN_0003.mp4,0,0,Na,Na
4,TRAIN_0004,./train/TRAIN_0004.mp4,1,1,0,0
...,...,...,...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3,1,1,0
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5,1,2,0
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0,0,Na,Na
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0,0,Na,Na


### 라벨링 기준 데이터 EDA

In [111]:
df['crash_ego'].value_counts()

0    1783
1     491
2     424
Name: crash_ego, dtype: int64

In [112]:
df['weather'].value_counts()

Na    1783
0      716
1      129
2       70
Name: weather, dtype: int64

In [113]:
df['timing'].value_counts()

Na    1783
0      808
1      107
Name: timing, dtype: int64

### 데이터 저장

In [127]:
crash_ego_df = df.drop(columns= ['label', 'weather', 'timing'])
crash_ego_df.to_csv('./data/train_crash_ego.csv', index=False)

In [130]:
weather_df = df.drop(columns= ['label', 'crash_ego', 'timing'])
weather_df.to_csv('./data/train_weather.csv', index=False)

In [131]:
timing_df = df.drop(columns= ['label', 'crash_ego', 'weather'])
timing_df.to_csv('./data/train_timing.csv', index=False)

### crash_ego_label df 편집

In [119]:
# data 경로 재지정, aug 대상여부 구분
for v in range(len(crash_ego_df['video_path'])):
    temp = crash_ego_df.loc[v,'video_path'].split('./')[-1]
    new_path = './data/' + temp
    crash_ego_df.loc[v,'video_path'] = new_path
crash_ego_df['aug'] = 'N'
crash_ego_df

,sample_id,video_path,crash_ego,aug
0,TRAIN_0000,./data/train/TRAIN_0000.mp4,2,N
1,TRAIN_0001,./data/train/TRAIN_0001.mp4,2,N
2,TRAIN_0002,./data/train/TRAIN_0002.mp4,0,N
3,TRAIN_0003,./data/train/TRAIN_0003.mp4,0,N
4,TRAIN_0004,./data/train/TRAIN_0004.mp4,1,N
...,...,...,...,...
2693,TRAIN_2693,./data/train/TRAIN_2693.mp4,1,N
2694,TRAIN_2694,./data/train/TRAIN_2694.mp4,1,N
2695,TRAIN_2695,./data/train/TRAIN_2695.mp4,0,N
2696,TRAIN_2696,./data/train/TRAIN_2696.mp4,0,N


In [120]:
# normal 라벨 기준 몇배 차이나는지 확인
crash_ego_label_aug_dict = {'Na': 0, 1: 3, 2: 3}

for i in tqdm(range(len(df))):
    crash_ego_label = crash_ego_df.loc[i, 'crash_ego']
    
    if (crash_ego_label != 0):
        for multi in range(crash_ego_label_aug_dict[crash_ego_label]):
            if(len(crash_ego_df[crash_ego_df['crash_ego']==crash_ego_label]) > 1783):
                break
            else:
                idx = len(crash_ego_df)
                id = 'TRAIN_aug' + '{0:04d}'.format(idx+i)
                vid_path = crash_ego_df.loc[i, 'video_path']
                is_aug = 'Y'
                crash_ego_df.loc[idx+i] = [id, vid_path, crash_ego_label, is_aug]     
        
crash_ego_df

  0%|          | 0/2698 [00:00<?, ?it/s]

,sample_id,video_path,crash_ego,aug
0,TRAIN_0000,./data/train/TRAIN_0000.mp4,2,N
1,TRAIN_0001,./data/train/TRAIN_0001.mp4,2,N
2,TRAIN_0002,./data/train/TRAIN_0002.mp4,0,N
3,TRAIN_0003,./data/train/TRAIN_0003.mp4,0,N
4,TRAIN_0004,./data/train/TRAIN_0004.mp4,1,N
...,...,...,...,...
7943,TRAIN_aug7943,./data/train/TRAIN_2685.mp4,2,Y
7944,TRAIN_aug7944,./data/train/TRAIN_2685.mp4,2,Y
7952,TRAIN_aug7952,./data/train/TRAIN_2692.mp4,2,Y
7953,TRAIN_aug7953,./data/train/TRAIN_2692.mp4,2,Y


In [122]:
crash_ego_df['crash_ego'].value_counts()

1    1784
0    1783
2    1696
Name: crash_ego, dtype: int64

In [126]:
crash_ego_df.to_csv('./data/train_crash_ego_aug.csv', index=False)